In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Let's prepare data Preparation
import nltk
nltk.download('names')
from nltk.corpus import names

male_names = names.words('male.txt')
female_names = names.words('female.txt')

data = [(name, 0) for name in male_names] + [(name, 1) for name in female_names]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Tokenization and Encoding

class NamesDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=32):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        name, label = self.data[idx]
        encoding = self.tokenizer(name, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

dataset = NamesDataset(data, tokenizer)

# Model Architecture
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Let's train the model
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(3):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

# Inference
def classify_name(name, model, tokenizer):
    encoding = tokenizer(name, return_tensors='pt')
    with torch.no_grad():
        output = model(**encoding)
    probabilities = torch.softmax(output.logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    if predicted_class == 0:
        return "Male"
    else:
        return "Female"

# Let's pick a random english name as example usage
name_to_classify = "John"
print(f"The name {name_to_classify} is classified as {classify_name(name_to_classify, model, tokenizer)}")

C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\ProgramData\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warn